In [14]:
COLLABORATORS = "Alexander Walla, Lennard Langenbruch, Marc-Alexander Richts"

# Intro

*Boosting* bezeichnet in Machine Learning einen "[ensemble](https://de.wikipedia.org/wiki/Ensemble_learning) [Metaalgorithmus](https://de.wikipedia.org/wiki/Metaheuristik)". Ensemble Learning verwendet mehrere Lernalgorithmen, um prädikative [Inferenz](https://de.wikipedia.org/wiki/Statistische_Inferenz) (Vorhersagen) im Vergleich zu einzelnen Lernalgorithmen zu verbessern. Boosting ist eine Art von Implementation von Ensemble Learning um mehrere schwache Klassifikatoren zu einem leistungsfähigen Klassifikator zusammenzufügen. Beim Boosting werden die Prädikatoren in "Reihe geschaltet" und nacheinander trainiert, sodass jeder Prädikator versucht die Fehler des vorherigen Prädikators zu beheben.

Die einzelnen schwachen Klassifikatoren müssen nur leicht besser als zufälliges raten sein.

# AdaBoost

AdaBoost (kurz für "Adaptive Boost") ist eine Boostingmethode, die den vorherigen Prädikator korrigiert, indem der nachfolgende Prädikator seinen Bias (Gewichte) zu den im Vorgängerprädikator nicht beachteten Trainingsdatenpunkten schiebt. Damit fokussieren sich die in der Kette später kommenden Prädikatoren mehr und mehr auf die schwierigeren Fälle. [1]

Je nach Genauigkeit der einzelnen Prädikatoren werden diese gewichtet. Je genauer ein Prädikator ist, desto höher ist sein Gewicht. Bei einer Genauigkeit, die zufälligem raten entspricht, bekommt der Prädikator ein Gewicht von ``0``. Wenn der Prädikator eine niedrigere Genauigkeit als 50% Trefferquote hat, bekommt er ein negatives Gewicht. [2]

**Es gibt also zwei Arten von Gewichten**:
- Die Trainingsdaten bekommen Gewichte $ 1 / m $. Zuerst sind diese Datenpunkte alle neutral $ m = 1 => 1 / 1 = 1 $ gewichtet, das bedeutet, der erste Prädikator wird auf die originalen Daten trainiert. Danach wird die gewichtete Fehlerquote auf die Trainingsdaten berechnet.
- Alle Prädikatoren bekommen je nach Genauigkeit ein hohes oder niedriges Gewicht. Dieses Gewicht wird am Ende im Ensemble verwendet, um eine Mehrheit auszurechnen. Das Gewicht eines Prädikators $ a_{j} $ berechnet sich wie folgt:

$$ a_{j} = n log \frac{1 - r_{j}}{r_{j}} $$
$$(Géron, 2020, p. 251)$$

- Der Hyperparameter $ n $ stellt hier die Lernrate dar, der standardmäßig auf 1 gesetzt ist.
- $ r_{j} $ beschreibt die Fehlerrate


Danach werden die falsch klassifizierten Datenpunkte *geboosted*; diese Datenpunkte bekommen ein höheres Gewicht. Wie oben bereits beschrieben, verschiebt sich dadurch der Bias auf die Datenpunkte auf die schwierigeren Cases.


## Vergleich von AdaBoost Prädikatoren zu Random Forests
Der Vergleich von AdaBoost zu Random Forests eignet sich sehr gut, da viele Merkmale dieser Algorithmen invers sind.
- Bei Random Forests hat jeder Tree (Prädikator) ein gleichwertiges Gewicht; bei AdaBoost bekommen die Prädikatoren ein verschieden schweres Gewicht wodurch einige Prädikatoren "mehr zu sagen" haben als andere.
- Bei Random Forests können die Trees parallel auf einzelne Merkmale trainiert werden. Bei AdaBoost werden die Learner jedoch sequenziell trainiert, wodurch sich das training nur sehr schwierig skalieren lässt. Denn je nachdem wie genau der erste Learner ist, werden die Datenpunkte für den zweiten Learner geboosted. Diese Abhängigkeit führt zu einer langen Kaskade.

## Applikation und
Eine der interessantesten Anwendungen von AdaBoost ist schnelle und effiziente Objektklassifizierung auf (aus der heutigen Sicht) Computern mit niedriger Rechenleistung. 2001 haben Viola und Jones auf einem 700MHz Pentium III auf einem 384 x 288 Pixel-Foto mit 15 Frames pro Sekunde klassifizieren können, ob das Bild ein Gesicht enthält oder nicht.
Bei Viola und Jones wird jeder schwache Lerner von AdaBoost auf ein einzelnes Merkmal trainiert. Dann wird der jeweilige Lerner mit der geringsten Fehlerrate genommen. Dadurch entsteht bereits im Trainingsschritt eine Art Selektion. Da Viola und Jones am Anfang der Klassifizierungskaskade weniger komplexe Learner verwenden, kann ein Gesicht im Klassifizierungsprozess bereits relativ früh abgelehnt werden, wodurch die späteren komplexeren Learner teilweise gar nicht erst aufgerufen werden - wodurch die Performance des Algorithmus im echten Leben gesteigert wird.

*Anmerkung: Der Teil vom Paper von Viola und Jones [5], der sich auf AdaBoost bezieht, ist sehr simpel und verständlich geschrieben. Für alle, die sich mehr mit diesem Thema beschäftigen wollen, und sogar Machine Learning auf embedded Systemen anwenden möchten, ist ein Blick auf das Paper ab Kapitel 3 "Learning Classification Functions" sehr viel wert.*

# AdaBoost in SciKit-Learn

SciKit-Learn implementiert eine angepasste Version von AdaBoost namens *Discrete SAMME*. SAMME steht für **S**tagewise **A**dditive **M**odeling using a **M**ulticlass **E**xponential loss function.
Diese angepasste Version nach dem Paper "Multi-class AdaBoost" [3] kann im Gegensatz zum originalen AdaBoost nicht nur binäre Entscheidungen treffen, sondern in mehr als nur zwei Kategorien unterscheiden.

Wenn man Prädikatoren verwendet, welche nicht nur binäre Entscheidungen, sondern Wahrscheinlichkeiten zurückgeben, kann man außerdem eine abgewandelte Form von SAMME namens *SAMME.R* verwenden, wobei das R für **R**eal steht.

Der Unterschied zwischen SAMME und SAMME.R ist dass SAMME sich auf der Grundlage von Fehlern in den vorhergesagten Class Labels sich anpasst, während SAMME.R die vorhergesagten Klassenwahrscheinlichkeiten verwendet. SAMME.R erzielt im Allgemeinen oft bessere Ergebnisse als SAMME.




# Praktische Anwendung anhand eines Beispiels

In [15]:
from sklearn import datasets

# Hinweise zur Praxis

- Die Lernrate von ``1`` ist laut Peter Prettenhofer und Noel Dawe [4] nicht unbedingt für SAMME und SAMME.R beidermaßen optimal.
- Wenn das Model zu Overfitting neigt, kann es helfen die Anzahl der Lerner ``n_estimators`` zu verringern.

# Literaturverzeichnis
[1] sklearn.ensemble.AdaBoostClassifier. (n.d.). Scikit-Learn. Retrieved June 3, 2022, from [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)
[2] Géron, A. (2020). Praxiseinstieg Machine Learning mit Scikit-Learn, Keras und TensorFlow: Konzepte, Tools und Techniken für intelligente Systeme (Aktuell zu TensorFlow 2) [E-book]. Dpunkt.Verlag GmbH.
[3] J. Zhu, H. Zou, S. Rosset, T. Hastie. “Multi-class AdaBoost”, 2009.
[4] Dawe, N., & Prettenhofer, P. (n.d.). Discrete versus Real AdaBoost. Scikit-Learn. Retrieved June 3, 2022, from [https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#preparing-the-data-and-baseline-models](https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#preparing-the-data-and-baseline-models)
[5] Viola, P., & Jones, M. (2001). Rapid Object Detection using a Boosted Cascade of Simple Features. University of Texas. Retrieved June 14, 2022, from [https://www.cs.utexas.edu/~grauman/courses/spring2007/395T/papers/viola_cvpr2001.pdf](https://www.cs.utexas.edu/~grauman/courses/spring2007/395T/papers/viola_cvpr2001.pdf)